# Puerto Rico Travel Planner AI Chatbot

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GOOGLE_PLACES_API_KEY = os.getenv('OUR_GOOGLE_PLACES_API_KEY')
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')


## Upload datasets to Pinecone

In [107]:
from pinecone import Pinecone, ServerlessSpec
import openai
import json
import time
from urllib.parse import urlparse, parse_qs
import requests


INDEX_NAME = "puerto-rico-travel"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
if INDEX_NAME not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(INDEX_NAME)

# Load data
with open("data/hotels.json", "r", encoding="utf-8") as f:
    hotels_data = json.load(f)

# Function to generate embeddings
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  # Updated to match the new API structure


def extract_place_id(url):
    """Extracts Google Maps Place ID from the given URL"""
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return query_params.get("query_place_id", [None])[0]  # Return first item or None

def fetch_google_reviews(place_id):
    """Fetch rating and review count from Google Places API"""
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,user_ratings_total&key={GOOGLE_PLACES_API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    if "result" in data:
        return {
            "name": data["result"].get("name", "N/A"),
            "rating": data["result"].get("rating", 0),  # Default 0 if missing
            "review_count": data["result"].get("user_ratings_total", 0)  # Default 0 if missing
        }
    return {"name": "N/A", "rating": 0, "review_count": 0}

# Store data in Pinecone
for item in hotels_data:
    combined_text = f"{item['name']} {item['type']} {item['region']} {item['location']['city']} {item['description']}"
    
    # Get the embedding for the combined text
    vector = get_embedding(combined_text)

    
    place_id = extract_place_id(item['contact']['google_maps_url'])
    # Get rating & review count
    place_data = fetch_google_reviews(place_id)
    
    # Prepare metadata for the upsert
    metadata = {
        "name": item["name"],
        "text": item["description"],
        "description": item["description"],
        "type": item["type"],
        "region": item["region"],
        "location": json.dumps(item["location"]),  # Keeping the full location metadata
        "contact": json.dumps(item["contact"]),
        "url": item["url"],
        "rating": place_data["rating"],
        "review_count": place_data["review_count"]

    }
    index.upsert(vectors=[(str(item["id"]), vector, metadata)])
    time.sleep(1)  # Prevent hitting API rate limits

print("Data uploaded to Pinecone successfully!")


Data uploaded to Pinecone successfully!


In [108]:
from pinecone import Pinecone, ServerlessSpec
import openai
import json
import time


INDEX_NAME = "puerto-rico-travel"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
if INDEX_NAME not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(INDEX_NAME)

# Load data
with open("data/food_drinks.json", "r", encoding="utf-8") as f:
    food_drinks = json.load(f)

# Function to generate embeddings
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  # Updated to match the new API structure

def extract_place_id(url):
    """Extracts Google Maps Place ID from the given URL"""
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return query_params.get("query_place_id", [None])[0]  # Return first item or None

def fetch_google_reviews(place_id):
    """Fetch rating and review count from Google Places API"""
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,user_ratings_total&key={GOOGLE_PLACES_API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    if "result" in data:
        return {
            "name": data["result"].get("name", "N/A"),
            "rating": data["result"].get("rating", 0),  # Default 0 if missing
            "review_count": data["result"].get("user_ratings_total", 0)  # Default 0 if missing
        }
    return {"name": "N/A", "rating": 0, "review_count": 0}

# Store data in Pinecone
for item in food_drinks:
    combined_text = f"{item['name']} {item['type']} {item['region']} {item['location']['city']} {item['description']}"
    
    # Get the embedding for the combined text
    vector = get_embedding(combined_text)

    
    place_id = extract_place_id(item['contact']['google_maps_url'])
    # Get rating & review count
    place_data = fetch_google_reviews(place_id)
    
    # Prepare metadata for the upsert
    metadata = {
        "name": item["name"],
        "text": item["description"],
        "description": item["description"],
        "type": item["type"],
        "region": item["region"],
        "location": json.dumps(item["location"]),  # Keeping the full location metadata
        "contact": json.dumps(item["contact"]),
        "url": item["url"],
        "rating": place_data["rating"],
        "review_count": place_data["review_count"]

    }
    index.upsert(vectors=[(str(item["id"]), vector, metadata)])
    time.sleep(1)  # Prevent hitting API rate limits

print("Data uploaded to Pinecone successfully!")


Data uploaded to Pinecone successfully!


In [109]:
from pinecone import Pinecone, ServerlessSpec
import openai
import json
import time


INDEX_NAME = "puerto-rico-travel"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
if INDEX_NAME not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(INDEX_NAME)

# Load data
with open("data/things_to_do.json", "r", encoding="utf-8") as f:
    to_do_data = json.load(f)

# Function to generate embeddings
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  # Updated to match the new API structure

def extract_place_id(url):
    """Extracts Google Maps Place ID from the given URL"""
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return query_params.get("query_place_id", [None])[0]  # Return first item or None

def fetch_google_reviews(place_id):
    """Fetch rating and review count from Google Places API"""
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,user_ratings_total&key={GOOGLE_PLACES_API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    if "result" in data:
        return {
            "name": data["result"].get("name", "N/A"),
            "rating": data["result"].get("rating", 0),  # Default 0 if missing
            "review_count": data["result"].get("user_ratings_total", 0)  # Default 0 if missing
        }
    return {"name": "N/A", "rating": 0, "review_count": 0}

# Store data in Pinecone
for item in to_do_data:
    combined_text = f"{item['name']} {item['type']} {item['region']} {item['location']['city']} {item['description']}"
    
    # Get the embedding for the combined text
    vector = get_embedding(combined_text)

    
    place_id = extract_place_id(item['contact'].get('google_maps_url'))
    # Get rating & review count
    place_data = fetch_google_reviews(place_id)
    
    # Prepare metadata for the upsert
    metadata = {
        "name": item["name"],
        "text": item["description"],
        "description": item["description"],
        "type": item["type"],
        "region": item["region"],
        "location": json.dumps(item["location"]),  # Keeping the full location metadata
        "contact": json.dumps(item["contact"]),
        "url": item["url"],
        "rating": place_data["rating"],
        "review_count": place_data["review_count"],
        "accommodations": json.dumps(item["accommodations"]),
        "photo": item["photo"]

    }
    index.upsert(vectors=[(str(item["id"]), vector, metadata)])
    time.sleep(1)  # Prevent hitting API rate limits

print("Data uploaded to Pinecone successfully!")


Data uploaded to Pinecone successfully!


## Testing data retival from pinecone

In [2]:
import openai
from pinecone import Pinecone

INDEX_NAME = "puerto-rico-travel"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to the index
index = pc.Index(INDEX_NAME)

# Function to get the embedding of the user's query
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  

# Function to query Pinecone and format the results
def search_pinecone(user_query, top_k=5):
    # Get the vector embedding for the user's query
    query_vector = get_embedding(user_query)
    
    # Query Pinecone for the top_k matches
    response = index.query(
        vector=query_vector, 
        top_k=top_k, 
        include_metadata=True
    )
    
    # Format the search results
    results = [
        {
            "description": match["metadata"].get("description", "Unknown"),  # Use 'values' as the description
            "name": match["metadata"].get("name", "Unknown"),
            "type": match["metadata"].get("type", "Unknown"),
            "region": match["metadata"].get("region", "Unknown"),
            "url": match["metadata"].get("url", ""),
            "rating": match["metadata"].get("rating", 0),
            "review_count": match["metadata"].get("review_count", 0),
        }
        for match in response["matches"]
    ]
    
    return results


In [3]:
def rank_results(results):
    """Sort results based on rating (primary) and review count (secondary)."""
    if not results:
        return []
    
    return sorted(
        results, 
        key=lambda x: (
            x.get("rating", 0),  # Primary: Higher rating first
            x.get("review_count", 0)  # Secondary: More reviews as tiebreaker
        ), 
        reverse=True  # Sort in descending order
    )

def format_results(results):
    """Format ranked search results into a readable string."""
    if not results:
        return "Sorry, I couldn't find anything matching your request."

    response = "Here are some recommendations:\n\n"
    for i, item in enumerate(results, 1):
        response += (
            f"{i}. **{item.get('name', 'Unknown')}** ({item.get('type', 'Unknown Type')} in {item.get('region', 'Unknown Location')})\n"
            f"   ⭐ {item.get('rating', 'N/A')} ({item.get('review_count', 'N/A')} reviews)\n"
            f"   {item.get('description', 'No description available.')}\n"
            f"   [More Info]({item.get('url', '#')})\n\n"
        )
    
    return response

# Example user input
user_input = "What are the best activities in San Juan?"
search_results = search_pinecone(user_input)

# Rank results before formatting
ranked_results = rank_results(search_results)

# Print formatted ranked results
print(format_results(ranked_results))


Here are some recommendations:

1. **VIP Adventures Puerto Rico** (['Kayaking', 'Biking', 'Group-Friendly Activities & Tours', 'SUP / Paddleboarding', 'Training & Teambuilding Companies', 'Snorkeling', 'Other Watersports'] in Metro)
   ⭐ 4.8 (1142.0 reviews)
   VIP Adventure has more than 20 years of experience in the field of touristic activities around the island and event planning for top companies in Puerto Rico. Our company concept is based on three important aspects: outstanding experience: unique activities, human and nature relations, and the best-personalized service to create your best tour experience in Puerto Rico.
   [More Info](https://www.discoverpuertorico.com/profile/vip-adventures-puerto-rico/9402)

2. **Jaime Benítez National Park** (['Track & Field Venues', 'Other Watersports'] in Metro)
   ⭐ 4.7 (1829.0 reviews)
   Laguna del Condado National Park is a very popular spot for all the family. Great place for a walk or a jog, you can also practice sportslike paddleboar

## Testing langchain

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Initialize the embedding model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# Connect Pinecone to LangChain
vector_store = Pinecone(index=index, embedding=embedding, text_key="text")

def rank_results(results):
    """Sort retrieved results by rating and review count."""
    if not results:
        return []

    return sorted(
        results,
        key=lambda x: (
            x[0].metadata.get("rating", 0),  # Primary: Higher rating first
            x[0].metadata.get("review_count", 0)  # Secondary: More reviews as tiebreaker
        ),
        reverse=True  # Sort in descending order
    )

def search_and_rank(query):
    """Run the retrieval chain, rank results, and return sorted responses."""
    results = vector_store.similarity_search_with_score(query, k=10)
    ranked_results = rank_results(results)  # Rank the retrieved results

    if not ranked_results:
        return "Sorry, no relevant results found."

    response = "Here are some top recommendations:\n\n"
    for i, (doc, score) in enumerate(ranked_results, 1):
        metadata = doc.metadata

        response += (
            f"{i}. **{metadata.get('name', 'Unknown')}** ({metadata.get('type', 'Unknown Type')} in {metadata.get('region', 'Unknown Location')})\n"
            f"   ⭐ {metadata.get('rating', 'N/A')} ({metadata.get('review_count', 'N/A')} reviews)\n"
            f"   {metadata.get('description', 'No description available.')}\n"
            f"   [More Info]({metadata.get('url', '#' )})\n\n"
        )
    
    return response

# Example queries with ranked results
queries = [
    "luxury beachfront hotel in San Juan",
    "What are the best activities in San Juan?",
    "Find me a hotel in Puerto Rico with a pool.",
    "What are some popular restaurants in San Juan?", 
    "What are some historical sites in San Juan?"
]

for query in queries:
    print(f"\nQuery: {query}\n")
    print(search_and_rank(query))



Query: luxury beachfront hotel in San Juan

Here are some top recommendations:

1. **La Rosa de los Vientos** (Hotel in Metro)
   ⭐ 4.9 (12.0 reviews)
   This beach-side villa was actually an abandoned house before the owner decided to turn it into a tropical paradise guesthouse. A pink and white color scheme gives the property a vibrant and calm vibe.
   [More Info](https://www.discoverpuertorico.com/profile/la-rosa-de-los-vientos/2411)

2. **The Ritz-Carlton San Juan Hotel** (Hotel in Metro)
   ⭐ 4.5 (863.0 reviews)
   Let one of the most elegant San Juan luxury resorts, The Ritz-Carlton, San Juan awaken your senses as you fall under the spell of its enchanting rhythms and flavors. Our casual yet elegant luxury hotel envelops guests in a timeless Caribbean setting – perched on a two-mile stretch of a golden sandy beach. Puerto Rican fritters, mamposteao rice and mofongo introduce adventurous palates to the local cuisine. Rum flights find their way on to afternoon agendas. And the pe

## Define Tools (Function Calling)

In [2]:
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import json
import requests
from geopy.distance import geodesic

In [ ]:
import openai
from pinecone import Pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

INDEX_NAME = "puerto-rico-travel"

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Connect to the index
index = pc.Index(INDEX_NAME)

# Initialize the embedding model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# Connect Pinecone to LangChain
vector_store = Pinecone(index=index, embedding=embedding, text_key="text") 

In [34]:
# Initialize user preferences storage
user_preferences = {
    "travel_dates": None,
    "interests": [],
    "locked_locations": [],
}

In [35]:
def find_weather_forecast(location: str):
    API_KEY = OPENWEATHER_API_KEY
    formatted_location = f"{location},PR,US"
    
    # Check if the user has travel dates stored
    travel_dates = user_preferences.get("travel_dates", "your trip")
    print(f"Checking weather for {formatted_location} on {travel_dates}.")

    url = f"https://api.openweathermap.org/data/2.5/weather?q={formatted_location}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        return f"Weather in {location} on {travel_dates}: {data['weather'][0]['description']}, {data['main']['temp']}°C, Humidity: {data['main']['humidity']}%"
    else:
        return f"Could not fetch weather for {location}."


In [36]:
def rank_appropriate_locations(user_prompt: str = None):
    # Use stored interests if no user input is provided
    if not user_prompt:
        interests = ", ".join(user_preferences.get("interests", []))
        user_prompt = f"Best places for {interests} in Puerto Rico"

    results = vector_store.similarity_search_with_score(user_prompt, k=5)

    ranked_results = sorted(results, key=lambda x: (
        x[0].metadata.get("rating", 0),
        x[0].metadata.get("review_count", 0)
    ), reverse=True)

    return [{"name": item[0].metadata["name"], "rating": item[0].metadata["rating"]} for item in ranked_results]


In [37]:
def find_info_on_location(location: str):
    results = vector_store.similarity_search_with_score(location, k=1)
    
    if not results:
        return "No information found."

    place = results[0][0].metadata

    # Parse `contact` field (since it's stored as a JSON string)
    contact_info = json.loads(place.get("contact", "{}"))  # Defaults to an empty dict if missing
    google_maps_url = contact_info.get("google_maps_url", "No directions available.")

    return f"{place['name']} - {place['description']} (Rating: {place.get('rating', 'N/A')}, {place.get('review_count', 'N/A')} reviews). More info: {place['url']} Directions: {google_maps_url}"


In [38]:
def compute_distance_to_list(new_location: str):
    locked_locations = user_preferences.get("locked_locations", [])
    
    if new_location not in location_coords:
        return f"Coordinates for {new_location} not found."

    new_location_coords = location_coords[new_location]
    distances = {}

    for loc in locked_locations:
        if loc in location_coords:
            distance_km = geodesic(location_coords[loc], new_location_coords).km
            distances[loc] = f"{distance_km:.2f} km"

    return distances if distances else "No locked locations to compare."


In [39]:
def itinerary_planner(days: int = 5):
    travel_dates = user_preferences.get("travel_dates", "upcoming trip")
    interests = user_preferences.get("interests", [])

    if not interests:
        return "Please provide your interests before generating an itinerary."

    user_query = f"Best places for {', '.join(interests)} in Puerto Rico"
    ranked_places = rank_appropriate_locations(user_query)

    if not ranked_places:
        return "I couldn't find enough locations matching your interests."

    daily_itinerary = {f"Day {i+1}": [] for i in range(days)}
    
    for i, place in enumerate(ranked_places):
        day_index = i % days  
        daily_itinerary[f"Day {day_index+1}"].append(place)

    itinerary = f"**Itinerary for {travel_dates}**\n\n"
    
    for day, places in daily_itinerary.items():
        itinerary += f"**{day}:**\n"
        for place in places:
            itinerary += f"  - {place['name']} (⭐ {place['rating']})\n"
        itinerary += "\n"

    return itinerary


In [40]:
def update_preferences(key, value):
    if key == "interests":
        if isinstance(value, str) and value not in user_preferences[key]:
            user_preferences[key].append(value)  # Append new interests
    elif key == "locked_locations":
        if isinstance(value, str) and value not in user_preferences[key]:
            user_preferences[key].append(value)  # Append new locations
    else:
        user_preferences[key] = value  # Store single-value fields

    return f"Updated {key}: {user_preferences[key]}"


In [57]:

def ask_travel_dates(_: str = None):
    return "What are your travel dates? I’ll remember them for later."

def set_travel_dates(dates: str):
    user_preferences["travel_dates"] = dates
    return f"Got it! Your travel dates are set to {dates}."

def ask_interests(_: str = None):
    return "What are your travel interests? (e.g., hiking, history, beaches)"

def set_interests(interests: list):
    user_preferences["interests"].extend(interests)
    return f"Added {', '.join(interests)} to your interests."



In [ ]:
def lock_location(location: str):
    user_preferences["locked_locations"].append(location)
    return f"{location} has been locked in your itinerary."


In [59]:
def chat_with_user(user_input):
    """Handle chatbot conversation flow"""
    
    if "travel dates" in user_input.lower():
        return ask_travel_dates()
    
    elif "interests" in user_input.lower():
        return ask_interests()
    
    elif "lock" in user_input.lower():
        location = user_input.split("lock ")[-1]  # Extract location from input
        return lock_location(location)
    
    elif "show itinerary" in user_input.lower():
        return f"Your itinerary: {user_preferences['locked_locations']}"
    
    else:
        return "I can help you plan your trip! Tell me your travel dates and interests."


In [60]:
from langchain.tools import Tool

# Define tools
tools = [
    Tool(
        name="itinerary_planner",
        func=itinerary_planner,
        description="Generates a travel itinerary using ranked RAG retrieval based on user interests and travel dates."
    ),
    Tool(
        name="find_weather_forecast",
        func=find_weather_forecast,
        description="Get the weather forecast for a specific location."
    ),
    Tool(
        name="rank_appropriate_locations",
        func=rank_appropriate_locations,
        description="Find the best locations based on user preferences."
    ),
    Tool(
        name="find_info_on_location",
        func=find_info_on_location,
        description="Retrieve details about a specific location."
    ),
    Tool(
        name="compute_distance_to_list",
        func=compute_distance_to_list,
        description="Calculate the distance from a new location to a list of locations."
    ),
    Tool(
        name="ask_travel_dates", 
        func=ask_travel_dates, 
        description="Ask user for travel dates."
        ),
    Tool(name="ask_interests", 
         func=ask_interests, 
         description="Ask user for travel interests."
         ),
    Tool(name="lock_location", 
         func=lock_location, 
         description="Lock a location in the itinerary."
         ),
    
]


In [61]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 🔹 Initialize OpenAI Model with Function Calling
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 🔹 Initialize Memory (Stores Conversation History)
memory = ConversationBufferMemory(
    memory_key="chat_history",  # This key must match the prompt variable
    return_messages=True
)

# 🔹 Define the Prompt Correctly (Now Integrated with Memory)
prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are 'The Hitchhiker's Guide to Puerto Rico', a travel assistant.
        Your job is to help users plan trips by:
        - Asking for travel dates and storing them
        - Asking for interests and storing them
        - Suggesting locations based on interests
        - Locking locations if the user confirms
        - Providing a finalized itinerary when asked

        Keep track of the conversation history and follow up where needed.
    """),
    ("placeholder", "{chat_history}"),  # Memory will be inserted here
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),  # This is needed for tool calls
])

# 🔹 Create the Agent with Function Calling
agent = create_tool_calling_agent(llm, tools, prompt)

# 🔹 Wrap in an Executor (Handles Conversation Memory & Tools)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True  # Avoids crashes if function calls fail
)



In [24]:
# Test the chatbot with various queries
queries = [
    "Hi",
    "I'm visiting Puerto Rico from March 10-15. I love historical sites and beaches. Can you create an itinerary?",
    "those locations seem spread out all over the island, can you create an itenerary with places closer to each other?",
    "What's the weather like in on those locations?",
    "what are the top beaches in Puerto Rico?",
    "give me more information about El Yunque National Forest",
    "Tell me about playa sucia.",
]

# Run the agent for each query
for query in queries:
    print(f"\n🗨️ **User:** {query}\n")
    response = agent_executor.invoke({"input": query})
    print(f"🤖 **Chatbot:** {response['output']}\n")



🗨️ **User:** Hi



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[5].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'tools[5].function.name', 'code': 'invalid_value'}}

In [67]:
response = agent_executor.invoke({"input": "I want to plan a trip to Puerto Rico!"})
print(response)
response = agent_executor.invoke({"input": "I'm visiting Puerto Rico from March 10-15."})
print(response)
response = agent_executor.invoke({"input": "I love historical sites and beaches. Can you create an itinerary?"})
print(response)
response = agent_executor.invoke({"input": "give me more locations in old san juan"})
print(response)




> Entering new AgentExecutor chain...
That sounds exciting! To help you plan your trip, I'll need to know a few details. First, could you tell me the dates you're planning to travel?

> Finished chain.
{'input': 'I want to plan a trip to Puerto Rico!', 'chat_history': [HumanMessage(content='I want to plan a trip to Puerto Rico!', additional_kwargs={}, response_metadata={}), AIMessage(content="That sounds exciting! To help you plan your trip, I'll need to know a few details. First, could you tell me the dates you're planning to travel?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I want to plan a trip to Puerto Rico!', additional_kwargs={}, response_metadata={}), AIMessage(content="That sounds exciting! To help you plan your trip, I'll need to know a few details. First, could you tell me the dates you're planning to travel?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm visiting Puerto Rico from March 10-15.", additional_kwargs={}, res

In [70]:
response = agent_executor.invoke({"input": "yes"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `lock_location` with `El Castillo San Felipe del Morro`


El Castillo San Felipe del Morro has been locked in your itinerary.El Castillo San Felipe del Morro has been locked into your itinerary. Would you like to add more locations or activities, or is there anything else you'd like to explore in Puerto Rico?

> Finished chain.
{'input': 'yes', 'chat_history': [HumanMessage(content='I want to plan a trip to Puerto Rico!', additional_kwargs={}, response_metadata={}), AIMessage(content="That sounds exciting! To help you plan your trip, I'll need to know a few details. First, could you tell me the dates you're planning to travel?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I want to plan a trip to Puerto Rico!', additional_kwargs={}, response_metadata={}), AIMessage(content="That sounds exciting! To help you plan your trip, I'll need to know a few details. First, could you tell me the dates you're planning to trav

In [ ]:
print(find_weather_forecast("2025-03-10", "San Juan"))
print(rank_appropriate_locations("I love hiking and history."))
print(find_info_on_location("Ropa Vieja Grill"))
print(compute_distance_to_list(["San Juan"], "Ponce"))


Weather in San Juan on 2025-03-10: clear sky, 42.12°C, Humidity: 12%
[{'name': 'Acampa Nature Adventures', 'rating': 4.7}, {'name': 'Go Hiking PR', 'rating': 4.7}, {'name': 'Pachamama Nature Park', 'rating': 0.0}, {'name': 'Tour Guide Debbie', 'rating': 0.0}, {'name': 'Create Adventure Puerto Rico', 'rating': 0.0}]
Ropa Vieja Grill - This popular restaurant exhibits Caribbean pride and flavor through their delicious Cuban Puerto Rican fusion. While visiting, try the veal tail in Cuban sauce or the "Island Mojo", a steak served in an exquisite house sauce. -Hours 11:30 a.m. - 10:30 p.m.. (Rating: 4.6, 3440.0 reviews). More info: https://www.discoverpuertorico.com/profile/ropa-vieja-grill/3136
{'San Juan': '73.63 km'}
